# Superday

In [37]:
import pandas as pd

In [38]:
df = pd.read_csv('Pre-Super_Day_candidate_dataset__28candidate_29.csv')
print(df.shape)
df.head(5)

(100000, 14)


,User ID,applications,Reason,Loan_Amount,FICO_score,Fico_Score_group,Employment_Status,Employment_Sector,Monthly_Gross_Income,Monthly_Housing_Payment,Ever_Bankrupt_or_Foreclose,Lender,Approved,bounty
0,00007820-89cb-4c1d-9940-eb270d605a35,1,cover_an_unexpected_cost,100000,669,fair,full_time,consumer_discretionary,5024,927,0,B,0,0
1,00012b55-514c-421e-9c76-3300abbc1134,1,credit_card_refinancing,70000,594,fair,full_time,information_technology,5764,1177,0,B,0,0
2,000157c1-b6a3-4c86-82c7-9ec1bda3799a,1,home_improvement,10000,596,fair,full_time,information_technology,4017,1487,0,A,0,0
3,00020400-efab-4b10-8812-2a0aaf774841,1,home_improvement,100000,642,fair,part_time,energy,3129,904,0,A,0,0
4,0002f737-0cda-48fb-91ed-533f3d0eab05,1,major_purchase,30000,642,fair,full_time,energy,4220,1620,0,A,0,0


## Feature Engineering

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [39]:
df['Fico_Score_group'].unique()

array(['fair', 'poor', 'good', 'very_good', 'excellent'], dtype=object)

In [40]:
df['Employment_Status'].unique()

array(['full_time', 'part_time', 'unemployed'], dtype=object)

In [55]:
# Encode ordinal categorical variables in order and non-ordinal with one hot encoding 

fico_group_map = { 'poor': 0, 'fair': 1, 'good': 2, 'very_good': 3, 'excellent': 4 }
employment_status_map = { 'unemployed': 0, 'part_time': 1, 'full_time': 2 }

df_encoded = df.copy()
df_encoded = pd.get_dummies(df_encoded, columns=['Reason'], prefix=['Reason'])
df_encoded = pd.get_dummies(df_encoded, columns=['Employment_Sector'], prefix=['Employment_Sector'])
df_encoded = pd.get_dummies(df_encoded, columns=['Lender'], prefix=['Lender'])

df_encoded["Fico_Score_group"] = df_encoded["Fico_Score_group"].map(fico_group_map)
df_encoded["Employment_Status"] = df_encoded["Employment_Status"].map(employment_status_map)
df_encoded.head(2)

,User ID,applications,Loan_Amount,FICO_score,Fico_Score_group,Employment_Status,Monthly_Gross_Income,Monthly_Housing_Payment,Ever_Bankrupt_or_Foreclose,Approved,...,Employment_Sector_financials,Employment_Sector_health_care,Employment_Sector_industrials,Employment_Sector_information_technology,Employment_Sector_materials,Employment_Sector_real_estate,Employment_Sector_utilities,Lender_A,Lender_B,Lender_C
0,00007820-89cb-4c1d-9940-eb270d605a35,1,100000,669,1,2,5024,927,0,0,...,False,False,False,False,False,False,False,False,True,False
1,00012b55-514c-421e-9c76-3300abbc1134,1,70000,594,1,2,5764,1177,0,0,...,False,False,False,True,False,False,False,False,True,False


## Tell us about the variables
_Possible things to consider: Which variables are the most helpful in understanding if a customer is going to be approved or denied for a loan? Are there certain variables that are not useful to collect?_

In [56]:
from sklearn.ensemble import RandomForestClassifier

In [75]:
features = df_encoded.drop(['Approved', 'bounty', 'User ID'], axis=1)  # Features
target = df_encoded['Approved']
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(features, target)

RandomForestClassifier(random_state=42)

In [58]:
feature_importances = rf_classifier.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': features.columns, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

display(feature_importance_df)

,Feature,Importance
2,FICO_score,0.222897
5,Monthly_Gross_Income,0.219216
6,Monthly_Housing_Payment,0.215286
1,Loan_Amount,0.094528
3,Fico_Score_group,0.050618
10,Reason_debt_conslidation,0.014148
4,Employment_Status,0.012721
8,Reason_cover_an_unexpected_cost,0.012512
9,Reason_credit_card_refinancing,0.012281
12,Reason_major_purchase,0.011976


# Takeaways
- The top most important features are, in order [FICO_score, Monthly_Gross_Income, Monthly_Housing_Payment]
- Employment Sector is generally not important an important factor in approval.

## Tell us about the lenders
_Possible things to consider: What is each Lender’s average approval rate? How does their revenue per approval differ? Are there any clear differences between the three different lenders on what type of customers they approve?_

Percent% approval by Vendor

In [9]:
df[['Lender', 'Approved']].groupby('Lender').mean() * 100

,Approved
Lender,
A,10.965455
B,7.127273
C,17.057143


In [24]:
approved = df[df['Approved'] == 1]

In [25]:
approved[(approved['Lender'] == 'A')].describe()

,applications,Loan_Amount,FICO_score,Monthly_Gross_Income,Monthly_Housing_Payment,Ever_Bankrupt_or_Foreclose,Approved,bounty
count,6031.0,6031.000000,6031.000000,6031.000000,6031.000000,6031.000000,6031.0,6031.0
mean,1.0,42417.509534,697.358481,7506.998674,1697.039794,0.005472,1.0,250.0
std,0.0,28226.467212,74.113488,3389.702307,670.735658,0.073775,0.0,0.0
min,1.0,5000.000000,353.000000,2007.000000,300.000000,0.000000,1.0,250.0
25%,1.0,20000.000000,659.500000,4857.500000,1263.000000,0.000000,1.0,250.0
50%,1.0,30000.000000,702.000000,7272.000000,1724.000000,0.000000,1.0,250.0
75%,1.0,70000.000000,739.000000,9612.000000,2109.500000,0.000000,1.0,250.0
max,1.0,100000.000000,850.000000,19997.000000,3299.000000,1.000000,1.0,250.0


In [27]:
approved[(approved['Lender'] == 'B')].describe()

,applications,Loan_Amount,FICO_score,Monthly_Gross_Income,Monthly_Housing_Payment,Ever_Bankrupt_or_Foreclose,Approved,bounty
count,1960.0,1960.000000,1960.000000,1960.000000,1960.000000,1960.0,1960.0,1960.0
mean,1.0,41790.816327,732.108673,8053.576020,1694.571939,0.0,1.0,350.0
std,0.0,27723.422857,53.841987,3256.772291,667.931901,0.0,0.0,0.0
min,1.0,5000.000000,600.000000,2014.000000,300.000000,0.0,1.0,350.0
25%,1.0,20000.000000,690.000000,5569.500000,1248.000000,0.0,1.0,350.0
50%,1.0,30000.000000,724.000000,7891.500000,1710.500000,0.0,1.0,350.0
75%,1.0,60000.000000,772.000000,10016.000000,2109.250000,0.0,1.0,350.0
max,1.0,100000.000000,850.000000,19909.000000,3287.000000,0.0,1.0,350.0


In [28]:
approved[(approved['Lender'] == 'C')].describe()

,applications,Loan_Amount,FICO_score,Monthly_Gross_Income,Monthly_Housing_Payment,Ever_Bankrupt_or_Foreclose,Approved,bounty
count,2985.0,2985.000000,2985.000000,2985.000000,2985.000000,2985.000000,2985.0,2985.0
mean,1.0,41135.678392,674.770519,6322.161809,1344.649581,0.016080,1.0,150.0
std,0.0,27824.071795,81.548255,3137.702364,672.804328,0.125806,0.0,0.0
min,1.0,5000.000000,358.000000,2008.000000,301.000000,0.000000,1.0,150.0
25%,1.0,20000.000000,622.000000,3936.000000,791.000000,0.000000,1.0,150.0
50%,1.0,30000.000000,682.000000,5634.000000,1260.000000,0.000000,1.0,150.0
75%,1.0,60000.000000,724.000000,8253.000000,1875.000000,0.000000,1.0,150.0
max,1.0,100000.000000,850.000000,19982.000000,3297.000000,1.000000,1.0,150.0


### Takeaways
- Lender C is the most likely to approve applications at 17% approval, more than twice as much as vendor B at 7%
- Lender B targets the 'Prime' sector, while Lender C targets the lowest earners and Lender A splits the difference. Approved applicants for Lender B have an average income of `8053` per month, compared to Lender C at `6322`. Average Fico score for lender B is `732` compared to C's average of `674`.
- Lender B never approves a person with a foreclosure
- The average loan amount approved by each vendor is roughly the same at ~`$41,000`
- Lender B pays the most for each conversion at `$350`, followed by Lender A at `$250` and Lender C at `150`


Lender B pays the most for each conversion, but also only approves "highest quality" candidate. Lender C approves more people but also pays less. Lender A splits the difference

## Tell us about which customers we should match to each lender

_Possible things to consider: Are there groups of customers that would be a better fit for a different lender? How much incremental revenue could we have made if we matched certain groups of customers more appropriately?_

### With a Model

In [129]:
# Total bounty with current selections
current_revenue = df['bounty'].sum()
current_revenue

2641500

In [95]:
predict_a = rf_classifier.predict_proba(features.assign(Lender_A= True, Lender_B=False, Lender_C=False))
predict_b = rf_classifier.predict_proba(features.assign(Lender_A= False, Lender_B=True, Lender_C=False))
predict_c = rf_classifier.predict_proba(features.assign(Lender_A= False, Lender_B=False, Lender_C=True))

In [108]:
lender_a = pd.Series(predict_a[:, 1] * 250, name='Lender_A')
lender_b = pd.Series(predict_b[:, 1] * 150, name='Lender_B')
lender_c = pd.Series(predict_c[:, 1] * 350, name='Lender_C')

predicted = pd.concat([lender_a, lender_b, lender_c], axis=1)
predicted.head(2)

,Lender_A,Lender_B,Lender_C
0,32.5,4.5,84.0
1,25.0,0.0,21.0


In [123]:
optimal_pick = predicted.idxmax(axis=1)
optimal_pick.head(3)

0    Lender_C
1    Lender_A
2    Lender_C
dtype: object

In [124]:

optimal_features = pd.get_dummies(optimal_pick)
optimal_features.head(5)

,Lender_A,Lender_B,Lender_C
0,False,False,True
1,True,False,False
2,False,False,True
3,False,False,True
4,False,False,True


In [130]:
with_optimal = features.assign(**optimal_features)
predicted_optimal_approval = rf_classifier.predict(with_optimal)
possible_revenue = optimal_pick.map({ 'Lender_A': 250, 'Lender_B': 350, 'Lender_C': 150 })

predicted_revenue = (predicted_optimal_approval * possible_revenue).sum()
predicted_revenue

2718650

In [131]:
predicted_revenue - current_revenue

77150

## Without a model

In [141]:
def group_to_lender(group):
  return {
    'poor': "Lender C",
    'fair': "Lender C",
    'good': "Lender A",
    'very_good': "Lender B",
    'excellent': "Lender B",
  }.get(group)

lender_by_group = df['Fico_Score_group'].map(group_to_lender)
lender_by_group.head(3)

0    Lender C
1    Lender C
2    Lender C
Name: Fico_Score_group, dtype: object

In [ ]:
a_proba = 

df 

## Takeaways

Lender C will approve more people but pays less, Lender A pays slightly more but approves fewer people, Lender B pays the most but approves the fewest people.

Generally we want to match users with a lower FICO score to Lender C, users with a average fico score to Lender A, and the highest average fico score to lender B.